In [1]:
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-2enjq36d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-2enjq36d
  Resolved https://github.com/huggingface/transformers.git to commit 7504be35ab88639ec3def32f0392f0a43c9551ee
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7191300 sha256=20a032381249e98cf5ca7dede9c7efbe2e93e2469bd672a098b6bd82df106e23
  Stored in directory: /tmp/pip-ephem-wheel-cache-f3nosw47/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers

In [2]:
pip install pytextrank

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import torch
import random
import spacy
import pytextrank
import logging
import wandb
from tqdm import tqdm
from torch.optim import Adam
from torch.nn import DataParallel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer,RobertaTokenizer, LongformerTokenizer,  BigBirdTokenizerFast
from transformers import BertForSequenceClassification, RobertaForSequenceClassification, LongformerForSequenceClassification, BigBirdForSequenceClassification
from transformers import TrainingArguments, Trainer

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
# Summarize the input to how many tokens, default to 512 for BERT use.
# Since LongFormer could accept 4096 tokens, we could skip TextRank if LongFormer
seed_val = 42
ENABLE_TEXT_RANK = False
TEXT_RANK_LENGTH = 512
LEARNING_RATE = 2e-5
NUM_EPOCHS = 4

MODEL = "longformer" # Choose from "bert", "roberta", "longformer", "bigbird"

TRAIN_DATA_PATH = "/kaggle/input/datasetfortextclassification/ds2_train.csv"
TEST_DATA_PATH = "/kaggle/input/datasetfortextclassification/ds2_test.csv"

DATASET = "ds1" if "ds1" in TRAIN_DATA_PATH else "ds2"
NEW_BATCH_SIZE = 4 if MODEL in ['longformer', 'bigbird'] else 8

wandb.login(key='bf24a38a046a0448057459477a5d48fbc6eb2f6a')
wandb.init()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fangkangmi. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20230615_102813-gtfep2cm
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run golden-sea-7
wandb: ⭐️ View project at https://wandb.ai/fangkangmi/uncategorized
wandb: 🚀 View run at https://wandb.ai/fangkangmi/uncategorized/runs/gtfep2cm


In [5]:
NEW_BATCH_SIZE

4

In [6]:
# If there's a GPU available...
# If GPU not available, training will cost SEVERAL DAYS, not recommended running on CPU
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU insteadp(not recommended).')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
# Set the seed value all over the place to make this reproducible.
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [8]:
train = pd.read_csv(TRAIN_DATA_PATH)
test = pd.read_csv(TEST_DATA_PATH)

In [9]:
print(train.shape)
train.head()

(4470, 3)


,label,text,label_id
0,Mortgage,we have been trying to get our home out of for...,0
1,Mortgage,carrington mortgage took over servicing my loa...,0
2,Mortgage,this is a dispute to case number . i received ...,0
3,Mortgage,"is to any ? dear .... my name is , with ...",0
4,Mortgage,", i took out a mortgage . less than a month ...",0


In [10]:
train["label"].value_counts()

Mortgage                   600
Credit card                600
Consumer Loan              600
Debt collection            600
Credit reporting           600
Bank account or service    600
Student loan               500
Money transfers            160
Prepaid card               130
Payday loan                 80
Name: label, dtype: int64

In [11]:
#Load the model, if no model to load, initialize all the model and tokenizer for further use.

checkpoint_path = "Model.pth"  # Change to your preferred location

# Suppress transformers warnings
logging.getLogger("transformers").setLevel(logging.ERROR)

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    losses = checkpoint['losses']
    print("Loaded checkpoint.")
else:
    print("Loading models...")
    tokenizer_options = {
        "bert": BertTokenizer.from_pretrained('bert-base-uncased'),
        "roberta": RobertaTokenizer.from_pretrained('roberta-base'),
        "longformer": LongformerTokenizer.from_pretrained('allenai/longformer-base-4096'),
        "bigbird":  BigBirdTokenizerFast.from_pretrained('google/bigbird-roberta-base')
    }


    
    model_options = {
        "bert": BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(train['label_id'].unique())),
        "roberta": RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(train['label_id'].unique())),
        "longformer": LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=len(train['label_id'].unique())),
        "bigbird": BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base', num_labels=len(train['label_id'].unique()))
    }
    print('done')
    

# Restore default logging level for transformers
logging.getLogger("transformers").setLevel(logging.WARNING)

Loading models...


done


By now, data has been cleaned. Next step is to tokenize, train, and test.
The first step is to use TextRank from Gensim to rank the first few sentences, and then we will train on BERT and RoBERTA to see its' performance.

# Text Rank by Spacy:
Note that this could be skipped.

In [12]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")

def summarize(input_text: str, max_tokens: int = 512):
    # Load a SpaCy model. For example, the English model.


    # Parse the document with SpaCy.
    doc = nlp(input_text)

    # If the text is less than 512 words, return the original text.
    if len(doc) <= max_tokens:
        return input_text

    # Extract the phrases using TextRank.
    phrases = ", ".join([p.text for p in doc._.phrases])

    summarized_doc = []  # Initialize an empty list to store the summarized sentences

    # Iterate over the sentences generated by textrank.summary and append them to the list
    for sent in doc._.textrank.summary(limit_sentences=50):
        summarized_doc.append(sent)


    # Join the sentences to form the summary.
    summary = " ".join([span.text for span in summarized_doc])

    # If the summary is longer than max_tokens, truncate it.
    if len(summary.split()) > max_tokens:
        summary = " ".join(summary.split()[:max_tokens])

    return summary

In [13]:
if (ENABLE_TEXT_RANK):
    print("\n-----------------------Progressing---------------------\n")
    try:
        train["text"] = train["text"].apply(summarize)
        test["text"] = test["text"].apply(summarize)
    except Exception as e:
        print("An error occurred while applying text_rank:", str(e))
    print("\n-----------------------Done---------------------\n")
else:
    print('TextRank has been skipped')

TextRank has been skipped


# Tokenize the data

In [14]:
train.head()

,label,text,label_id
0,Mortgage,we have been trying to get our home out of for...,0
1,Mortgage,carrington mortgage took over servicing my loa...,0
2,Mortgage,this is a dispute to case number . i received ...,0
3,Mortgage,"is to any ? dear .... my name is , with ...",0
4,Mortgage,", i took out a mortgage . less than a month ...",0


In [15]:
train_text = train.text.values
train_labels = train.label_id.values

test_text = test.text.values
test_labels = test.label_id.values

In [16]:
train_text.shape

(4470,)

In [17]:
# Use the selected tokenizer
tokenizer = tokenizer_options.get(MODEL)

if MODEL in ["bert", "roberta"]:
    MODEL_MAX_TOKEN = 512
elif MODEL in ["longformer", "bigbird"]:
    MODEL_MAX_TOKEN = 1024
else:
    raise ValueError("Invalid model specified.")

# Tokenize the text
def tokenize_text(text, max_length=MODEL_MAX_TOKEN):
    return tokenizer.batch_encode_plus(
        text.tolist(), 
        max_length=max_length, 
        padding='max_length', 
        truncation=True
    )

tokenized_train_texts = tokenize_text(train_text)
tokenized_test_texts = tokenize_text(test_text)

In [18]:
class TextClassificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the dataset
train_dataset = TextClassificationDataset(tokenized_train_texts, train['label_id'].values)
test_dataset = TextClassificationDataset(tokenized_test_texts, test['label_id'].values)

# Create the dataloader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [19]:
for i in range(len(train['label_id'].values)):
    if (train['label_id'].values[i]!=train_dataset[i]['labels']):
        print('not equal')
        

print(train['label_id'].values[500])
print('-----------------------------')
print(train_dataset[500])
print('-----------------------------')
for batch in train_dataloader:
    print(batch)
    break


0
-----------------------------
{'input_ids': tensor([   0,  627, 5501,  ...,    1,    1,    1]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'labels': tensor(0)}
-----------------------------
{'input_ids': tensor([[    0, 19491,   107,  ...,     1,     1,     1],
        [    0,  1469,   927,  ...,     1,     1,     1],
        [    0,   118,   524,  ...,     1,     1,     1],
        ...,
        [    0,  5760,     9,  ...,     1,     1,     1],
        [    0,   118,  1357,  ...,     1,     1,     1],
        [    0,   118,  1357,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([3, 2, 6, 8, 0, 0, 6, 1])}


# train the model

In [20]:
# Use the selected model
torch.cuda.empty_cache()
model = model_options.get(MODEL)
model = model.to('cuda')

training_args = TrainingArguments(
    output_dir='./results',  # Directory to save checkpoints and final model
    num_train_epochs= NUM_EPOCHS,  # Number of training epochs
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size=NEW_BATCH_SIZE,
    save_strategy='epoch',
    save_total_limit = 1,
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=200,
    evaluation_strategy='no',  # No evaluation during training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)
trainer.train()


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
200,1.343700
400,0.762000
600,0.608000
800,0.477800
1000,0.477100
1200,0.338300
1400,0.302500
1600,0.315800
1800,0.211400
2000,0.203700


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=2236, training_loss=0.47463324415449165, metrics={'train_runtime': 4412.6241, 'train_samples_per_second': 4.052, 'train_steps_per_second': 0.507, 'total_flos': 1.174518264250368e+16, 'train_loss': 0.47463324415449165, 'epoch': 4.0})

In [21]:
# Extract the predicted labels and true labels from the evaluation results
y_pred = trainer.predict(test_dataset).predictions.argmax(axis=1)
y_true = test_dataset.labels

# Print the classification report
report = classification_report(y_true, y_pred)
print(report)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


              precision    recall  f1-score   support

           0       0.98      0.94      0.96      5442
           1       0.86      0.88      0.87      1693
           2       0.60      0.82      0.69       466
           3       0.89      0.82      0.86      2232
           4       0.90      0.87      0.89      1153
           5       0.71      0.93      0.81       234
           6       0.84      0.85      0.84      1330
           7       0.31      0.76      0.44        41
           8       0.27      0.50      0.35        20
           9       0.33      0.87      0.47        30

    accuracy                           0.89     12641
   macro avg       0.67      0.82      0.72     12641
weighted avg       0.90      0.89      0.89     12641



# Below are the code to fintune Model by using torch 

In [22]:
'''
# Use the selected model
model = model_options.get(MODEL)

# Initialize the optimizer
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

# Move the model to the GPU
model.train()
model.to('cuda')

# Initialize a list to store the loss values
losses = []

checkpoint_path = f"{MODEL}_{DATASET}_TextRank_{ENABLE_TEXT_RANK}.pth" # Change to your preferred location

start_epoch = 0

# Load from checkpoint if it exists
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    losses = checkpoint['losses']
    print("Loaded checkpoint.")

# Training 
for epoch in range(start_epoch, NUM_EPOCHS):
    print(f"Starting epoch {epoch}")
    for batch in tqdm(train_dataloader, desc="Training"):
        batch = {k: v.to('cuda') for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        # Optimize the model parameters
        optimizer.step()
        optimizer.zero_grad()

        # Append the loss value to the list
        losses.append(loss.item())  

    # Save checkpoint after each epoch
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'losses': losses,
    }, checkpoint_path)
    print(f"Saved checkpoint for epoch {epoch}.")

    # Plot the loss values after each epoch
    plt.plot(losses)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Loss over Iterations')
    plt.show()

'''

'\n# Use the selected model\nmodel = model_options.get(MODEL)\n\n# Initialize the optimizer\noptimizer = Adam(model.parameters(), lr=LEARNING_RATE)\n\n# Move the model to the GPU\nmodel.train()\nmodel.to(\'cuda\')\n\n# Initialize a list to store the loss values\nlosses = []\n\ncheckpoint_path = f"{MODEL}_{DATASET}_TextRank_{ENABLE_TEXT_RANK}.pth" # Change to your preferred location\n\nstart_epoch = 0\n\n# Load from checkpoint if it exists\nif os.path.exists(checkpoint_path):\n    checkpoint = torch.load(checkpoint_path)\n    model.load_state_dict(checkpoint[\'model_state_dict\'])\n    optimizer.load_state_dict(checkpoint[\'optimizer_state_dict\'])\n    start_epoch = checkpoint[\'epoch\']\n    losses = checkpoint[\'losses\']\n    print("Loaded checkpoint.")\n\n# Training \nfor epoch in range(start_epoch, NUM_EPOCHS):\n    print(f"Starting epoch {epoch}")\n    for batch in tqdm(train_dataloader, desc="Training"):\n        batch = {k: v.to(\'cuda\') for k, v in batch.items()}\n\n        o

Model now been trained, now evaluate the performance

In [23]:
'''
#If yo have a trained model just want to evaluate it:
checkpoint_path = "Model.pth"  # Change to your preferred location
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    losses = checkpoint['losses']
    print("Loaded checkpoint.")
    

model.eval()
model.to('cuda')
predictions = []
true_labels = []

# Iterate over the test dataset
for batch in test_dataloader:
    batch = {k: v.to('cuda') for k, v in batch.items()}

    # Disable gradient calculation
    with torch.no_grad():
        # Perform inference
        outputs = model(**batch)
        
    predicted_values = outputs.logits
    predictions.extend(predicted_values)
    true_labels.extend(batch['labels'].tolist())

# Convert logits to predictions
predictions = [torch.argmax(item).item() for item in predictions]

# Make sure that true_labels is a list
assert isinstance(true_labels, list)

  
# Generate classification report
report = classification_report(true_labels, predictions)

print(report)

# Save the report to a file
with open(f'Classreport_{MODEL}_{DATASET}_TextRank_{ENABLE_TEXT_RANK}.txt', 'w') as file:
    file.write(report)

'''

'\n#If yo have a trained model just want to evaluate it:\ncheckpoint_path = "Model.pth"  # Change to your preferred location\nif os.path.exists(checkpoint_path):\n    checkpoint = torch.load(checkpoint_path)\n    model.load_state_dict(checkpoint[\'model_state_dict\'])\n    optimizer.load_state_dict(checkpoint[\'optimizer_state_dict\'])\n    start_epoch = checkpoint[\'epoch\']\n    losses = checkpoint[\'losses\']\n    print("Loaded checkpoint.")\n    \n\nmodel.eval()\nmodel.to(\'cuda\')\npredictions = []\ntrue_labels = []\n\n# Iterate over the test dataset\nfor batch in test_dataloader:\n    batch = {k: v.to(\'cuda\') for k, v in batch.items()}\n\n    # Disable gradient calculation\n    with torch.no_grad():\n        # Perform inference\n        outputs = model(**batch)\n        \n    predicted_values = outputs.logits\n    predictions.extend(predicted_values)\n    true_labels.extend(batch[\'labels\'].tolist())\n\n# Convert logits to predictions\npredictions = [torch.argmax(item).item() 